# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Impor the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [2]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

Number of cities in the list: 599


---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [10]:
# set base url
baseweatherUrl = "http://api.openweathermap.org/data/2.5/"
basegeoUrl = "https://api.geoapify.com/v2/"

weatherKey = weather_api_key
geoKey = "59627f8b9e9a4c24931c6dbde5be5529"

def getGeoData(city, country, apikey):
    endpoint = f"https://api.geoapify.com/v1/geocode/search?apiKey={apikey}&format=json&text={city}_{country}"
    return endpoint

# print(geoKey)

In [ ]:
weatherBaseUrl = f"{baseweatherUrl}weather?appid={weatherKey}"

url = f"{weatherBaseUrl}&q=Corona"

response = requests.get(url).json()

# url = "https://api.openweathermap.org/data/2.5/weather?appid=ccf7ac95d3227a8e645528f05592bb2a&q=Irvine"

response

In [11]:
placesBaseUrl = f"{basegeoUrl}places?apiKey={geoKey}"

url = getGeoData("Irvine","California", geoKey) # f"{placesBaseUrl}&categories=commercial"

url

'https://api.geoapify.com/v1/geocode/search?apiKey=59627f8b9e9a4c24931c6dbde5be5529&format=json&text=Irvine_California'